In [1]:
from langchain_community.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain.docstore.document import Document
from langchain.schema import HumanMessage
from langchain_community.chat_models import ChatOllama
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser, CommaSeparatedListOutputParser
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.runnable import RunnableLambda
from operator import itemgetter

import json
from langchain.docstore.document import Document


# Data Preparation

## Import scraped data from scraper

In [2]:
# from scraper import json_list

In [3]:
# json_list[:5]

In [4]:
# # เซฟทั้งหมดเป็นไฟล์เดียว
# with open("data.json", "w", encoding="utf-8") as f:
#     json.dump(json_list, f, ensure_ascii=False, indent=2)


## อ่านข้อมูลจาก json มาเก็บในลิสต์

In [2]:
json_list_x = []
with open("data.json", "r", encoding="utf-8") as f:
    json_list_x = json.load(f)


## Change json_list to Document

In [3]:
documents = []
for item in json_list_x:
    content = f"subject: {item['subject']}\n gender: {item['gender']}\n age: {item['age']}\n symptoms: {', '.join(item['symptoms'])}\n question: {item['question']}\n answer: {item['answer']}"
    documents.append(Document(page_content=content, metadata={"source": "scraper"}))



## Embeding and Store to VectorDB

In [4]:
embedder = OllamaEmbeddings(model="bge-m3")


C:\Users\Thirachai\AppData\Local\Temp\ipykernel_25228\2229192303.py:1: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embedder = OllamaEmbeddings(model="bge-m3")


In [5]:
#vector_store = FAISS.from_documents(documents, embedder)

vector_store = FAISS.load_local("D:/faiss_storage", 
                                embedder,
                                allow_dangerous_deserialization=True
)
print("Loaded vector store from disk")

Loaded vector store from disk


In [6]:
retriever = vector_store.as_retriever()



# Create chat

## Import llm

In [7]:
llm = ChatOllama(model="scb10x/llama3.1-typhoon2-8b-instruct",
                    temperature=0.4)
# from medpalm.model import MedPalm
# llm = MedPalm()
print(llm)

model='scb10x/llama3.1-typhoon2-8b-instruct' temperature=0.4


C:\Users\Thirachai\AppData\Local\Temp\ipykernel_25228\2354700130.py:1: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  llm = ChatOllama(model="scb10x/llama3.1-typhoon2-8b-instruct",


In [ ]:
# llm_2 = ChatOllama(model="koesn/llama3-openbiollm-8b",
#                     temperature=0.4)
# print(llm_2)

# from huggingface_hub import hf_hub_download
# from llama_cpp import Llama

# model_name = "aaditya/OpenBioLLM-Llama3-8B-GGUF"
# model_file = "openbiollm-llama3-8b.Q5_K_M.gguf"

# model_path = hf_hub_download(model_name,
#                              filename=model_file,
#                              local_dir='/content')
# print("My model path: ", model_path)
# llm_2 = Llama(model_path=model_path,
#             n_gpu_layers=-1,
#             temperature=0.5,
#             max_tokens=4096,
#             n_ctx=8000)

model='koesn/llama3-openbiollm-8b' temperature=0.4


## Create prompt template

In [13]:
prompt = ChatPromptTemplate.from_messages([
    ("system", 
     "คุณคือผู้แพทย์เชี่ยวชาญด้านสุขภาพที่มีความรู้ลึกซึ้งทั้งจากประสบการณ์และฐานข้อมูลทางการแพทย์ "
     "คุณจะต้องให้คำแนะนำที่แม่นยำและปลอดภัย โดยเริ่มต้นด้วยประโยค 'คำตอบจากแพทย์ผู้เชี่ยวชาญ' "
     "จากนั้นให้คำตอบแบ่งเป็นหัวข้อ: สาเหตุที่เป็นไปได้, แนวทางการดูแลเบื้องต้น, คำแนะนำเพิ่มเติม "
     " ใส่ข้อความ 'คนไข้สามารถค้นหาแพทย์เฉพาะทางที่ต้องการและทำนัดหมายได้ทางเว็บไซต์ของเราที่ลิ้งค์นี้ \nhttps://bit.ly/doctor-refer-agnos' ในช่วงท้ายของการสนทนา"),
    
    ("human", 
     "มีผู้ป่วยที่มีข้อมูลดังนี้:\n"
     "- หัวข้อ: {subject}\n"
     "- เพศ: {gender}\n"
     "- อายุ: {age} ปี\n"
     "- อาการ: {symptoms}\n"
     "- คำถาม: {question}\n"
     "ข้อมูลจากฐานข้อมูลที่เกี่ยวข้อง:\n{context}\n"
     "กรุณาวิเคราะห์และให้คำตอบอย่างละเอียด โดยอ้างอิงจากทั้งความรู้ของคุณและข้อมูลที่ให้มาในส่วนของ context อย่างรอบคอบ"
     "หากข้อมูลใน context ไม่เพียงพอ ให้ใช้ความรู้ทั่วไปของคุณในการตอบคำถามอย่างระมัดระวังและแนะนำให้ผู้ป่วยไปพบแพทย์ผู้เชี่ยวชาญ"
     )
])

In [14]:
from langchain.schema.runnable import RunnableLambda

def fill_missing_fields(inputs):
    return {
        "subject": inputs.get("subject", "ไม่ระบุ"),
        "gender": inputs.get("gender", "ไม่ระบุ"),
        "age": inputs.get("age", "ไม่ระบุ"),
        "symptoms": inputs.get("symptoms", "ไม่ระบุ"),
        "question": inputs.get("question", "ไม่ระบุ"),
        "context": inputs.get("context", "ไม่มีข้อมูลจากฐานข้อมูล")
    }

input_preprocessor = RunnableLambda(fill_missing_fields)


In [15]:
rag_chain = (
    {
        "context": retriever,
        "subject": RunnablePassthrough(),
        "gender": RunnablePassthrough(),
        "age": RunnablePassthrough(),
        "symptoms": RunnablePassthrough(),
        "question": RunnablePassthrough(),

    }
    | input_preprocessor
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
# rag_chain_2 = (
#     {
#         "context": retriever,
#         "subject": RunnablePassthrough(),
#         "gender": RunnablePassthrough(),
#         "age": RunnablePassthrough(),
#         "symptoms": RunnablePassthrough(),
#         "question": RunnablePassthrough(),

#     }
#     | input_preprocessor
#     | prompt
#     | llm_2
#     | StrOutputParser()
# )

In [16]:
data_input =[ {
    "subject": "อาการปวดหัวเรื้อรัง",
    "gender": "หญิง",
    "age": "35",
    "symptoms": "ปวดหัวบริเวณขมับทั้งสองข้างมาเกือบทุกวัน โดยเฉพาะช่วงบ่าย รู้สึกตึงๆ และมีอาการคลื่นไส้ร่วมด้วย",
    "question": "อาการแบบนี้เกิดจากอะไร และควรดูแลตัวเองอย่างไร?"
},
{
    "subject": "การนอนหลับในผู้สูงอายุ",
    "gender": "ชาย",
    "age": "68",
    "symptoms": "นอนหลับไม่สนิท ตื่นบ่อยตอนกลางคืน และรู้สึกไม่สดชื่นตอนเช้า",
    "question": "ควรปรับพฤติกรรมหรือมีวิธีดูแลสุขภาพอย่างไรให้หลับดีขึ้น?"
},
{
    "subject": "ภาวะขาดวิตามินบี 12 (Vitamin B12 deficiency)",
    "gender": "หญิง",
    "age": "29",
    "symptoms": [
      "เหนื่อยง่าย",
      "มือเท้าชา",
      "เวียนหัว",
      "ลืมง่าย",
      "อ่อนแรง"
    ],
    "question": "ช่วงนี้รู้สึกเหนื่อยง่ายมากค่ะ เดินนิดเดียวก็หอบ มือเท้าชาบ่อย ๆ โดยเฉพาะตอนตื่นนอน รู้สึกเวียนหัวและลืมง่ายขึ้นเรื่อย ๆ อยากทราบว่าอาการแบบนี้เกิดจากอะไร และควรตรวจอะไรบ้างคะ?"
  },
  {
    "subject": "โรคกรดไหลย้อน (GERD)",
    "gender": "ชาย",
    "age": "34",
    "symptoms": [
      "แสบร้อนกลางอก",
      "เรอเปรี้ยว",
      "ไอเรื้อรัง",
      "แน่นหน้าอก",
      "กลืนลำบาก"
    ],
    "question": "ผมมีอาการแสบร้อนกลางอกหลังทานอาหาร โดยเฉพาะอาหารเผ็ดหรือมัน ๆ เรอเปรี้ยวบ่อยมาก และบางครั้งรู้สึกแน่นหน้าอกจนหายใจไม่สะดวก อยากทราบว่าเป็นกรดไหลย้อนหรือเปล่า และควรปรับพฤติกรรมยังไงครับ?"
  },
  {
    "subject": "ภาวะถุงน้ำรังไข่หลายใบ (PCOS)",
    "gender": "หญิง",
    "age": "26",
    "symptoms": [
      "ประจำเดือนมาไม่ปกติ",
      "สิวขึ้นเยอะ",
      "ขนดก",
      "น้ำหนักขึ้นง่าย",
      "ปวดท้องน้อย"
    ],
    "question": "ประจำเดือนมาไม่ตรงเวลาเลยค่ะ บางเดือนก็ไม่มาเลย สิวขึ้นเยอะมากทั้งที่ดูแลผิวดีแล้ว ขนตามตัวก็ดูเยอะขึ้น น้ำหนักขึ้นง่ายมากทั้งที่กินเท่าเดิม อยากทราบว่าอาการแบบนี้เกี่ยวกับฮอร์โมนหรือเปล่า และควรไปตรวจอะไรบ้างคะ?"
  },
  {
    "subject": "โรคภูมิแพ้อากาศ (Allergic Rhinitis)",
    "gender": "ชาย",
    "age": "18",
    "symptoms": [
      "จามบ่อย",
      "คัดจมูก",
      "น้ำมูกไหล",
      "คันจมูก",
      "ตาแดง"
    ],
    "question": "ผมมีอาการจามบ่อยมาก โดยเฉพาะตอนเช้า ๆ หรือเวลาอยู่ในห้องแอร์ น้ำมูกไหล คัดจมูก และบางครั้งตาแดงคันร่วมด้วย อยากทราบว่าเป็นภูมิแพ้อากาศหรือเปล่า และมีวิธีดูแลตัวเองยังไงบ้างครับ?"
  }
]

In [17]:
my_question = rag_chain.invoke({    
    "subject": "กรดไหลย้อน อาหารไม่ย่อย",
    "gender": "ชาย",
    "age": "24",
    "symptoms": "แสบอก, แสบคอ, ท้องอืด, อาหารไม่ย่อย, จุกคอ",
    "question": "มีคืนนึงตอนผมกำลังจะนอนจู่ๆผมก็รู้สึกแสบคอขึ้นมา แล้วพอตื่นมาก็รู้สึกว่าเหมือนมีก้อนอะไรมาจุกที่คอ ผมเลยค้นหาจากอินเตอร์เน็ตก็พบว่าผมเป็นกรดไหลย้อนจึงไปซื้อยาที่ร้านขายยามากินเป็นเวลา 1 อาทิตย์ จนพอผมเริ่มรู้สึกดีขึ้นผมจึงไปกินบุฟเฟ่ต์ชาบูแล้วทีนี้ผมรู้สึกว่าอาหารมันไม่ย่อยเลยแม้แต่ตอนกำลังจะนอนก็รู้สึกได้ว่ามีอาหารอยู่ในท้องจนถึงวัดถัดไป ควรรักษาต่อยังไงดีครับ"
})

print(my_question)

คำตอบจากแพทย์ผู้เชี่ยวชาญ

สาเหตุที่เป็นไปได้:
อาการแสบอก, แสบคอ, ท้องอืด, อาหารไม่ย่อย, จุกคอที่ผู้ป่วยมี อาจเกิดจากกรดไหลย้อน (GERD) ซึ่งเป็นสภาวะที่กรดในกระเพาะอาหารไหลย้อนขึ้นไปในหลอดอาหาร ทำให้เกิดอาการแสบคอและจุกแน่นท้องได้

แนวทางการดูแลเบื้องต้น:
1. ทานยาลดกรด: เช่น omeprazole หรือ ranitidine เพื่อลดปริมาณกรดในกระเพาะอาหาร
2. ปรับพฤติกรรมการทานอาหาร: หลีกเลี่ยงการกินอาหารที่มีไขมันสูง, เผ็ด, อาหารรสเปรี้ยวหรือเปรี้ยวจัด, น้ำอัดลม และกาแฟ
3. รักษาน้ำหนักให้เหมาะสม: การมีน้ำหนักเกินอาจทำให้เกิดกรดไหลย้อนได้ง่ายขึ้น
4. หลีกเลี่ยงการนอนหลังทานอาหาร: ควรเว้นระยะเวลาอย่างน้อย 2-3 ชั่วโมงระหว่างการทานอาหารกับการนอน

คำแนะนำเพิ่มเติม:
หากอาการไม่ดีขึ้นภายใน 1-2 สัปดาห์ หรือมีอาการรุนแรงขึ้น เช่น ไอเรื้อรัง, แสบคอหรือเจ็บหน้าอกอย่างต่อเนื่อง ควรไปพบแพทย์เพื่อทำการตรวจวินิจฉัยและรับการรักษาที่เหมาะสม

คนไข้สามารถค้นหาแพทย์เฉพาะทางที่ต้องการและทำนัดหมายได้ทางเว็บไซต์ของเราที่ลิ้งค์นี้ 
https://bit.ly/doctor-refer-agnos


In [ ]:
my_question = rag_chain.invoke({    
    "question": "สวัสดีครับคือช่วงนี้ผมนอนไม่ค่อยหลับเวลานอนมันก็จะคิดฟุ้งซ่านตลอดเวลาหรือบางทีก็มีเพลงเล่นในหัวตลอดเวลาเลยทำให้นอนไม่พอ พอจะมีวิธีไหนช่วยจัดการไหมครับ"
})

print(my_question)

คำตอบจากแพทย์ผู้เชี่ยวชาญ

สาเหตุที่เป็นไปได้:
อาการนอนไม่หลับอาจเกิดจากหลายสาเหตุ เช่น ความเครียด, โรคทางจิตเวชบางอย่าง, หรือปัญหาทางกายภาพที่ทำให้เกิดการรบกวนในการนอนหลับ

แนวทางการดูแลเบื้องต้น:
1. จัดห้องนอนให้มีบรรยากาศเงียบสงบ ไม่มีแสงเสียงรบกวน และมีอากาศถ่ายเทสะดวก
2. ใช้ห้องนอนเพื่อการนอนเท่านั้น ไม่ใช้ห้องนอนทำกิจกรรมอื่น เช่น ดูทีวี ทำงาน อ่านหนังสือ
3. ก่อนนอน 1 ชั่วโมงควรมีเวลาให้ตัวเองได้พักผ่อนสบายๆ พยายามไม่คิดเรื่องเครียด เรื่องกังวลต่างๆ
4. เข้านอนเมื่อง่วงนอนเท่านั้น
5. ถ้าหากเข้านอนไปนาน 15-20 นาทีแล้วยังนอนไม่หลับก็ควรลุกขึ้นมาจากที่นอนเพื่อหากิจกรรมเบาๆทำก่อน แล้วกลับเข้าไปนอนเมื่อรู้สึกง่วงนอนอีกครั้ง
6. ตื่นนอนให้เป็นเวลาเดียวกันของทุกวัน
7. ออกกำลังกายอย่างสม่ำเสมอ แต่ไม่ควรออกกำลังกายหักโหมก่อนนอนเพราะจะทำให้นอนไม่หลับมากขึ้น
8. ไม่นอนกลางวัน
9. ไม่ดื่มชา กาแฟ หลังเที่ยงวัน
10. ไม่สูบบุหรี่หรือดื่มสุราก่อนนอน

คำแนะนำเพิ่มเติม:
หากปฏิบัติตามแนวทางการดูแลเบื้องต้นแล้ว 1-2 สัปดาห์การนอนยังไม่ดีขึ้น ควรไปพบจิตแพทย์เพื่อตรวจประเมินอาการเพิ่มเติมและพิจารณาการรักษาท

In [ ]:
# my_question_x2 = rag_chain_2.invoke({    
#     "question": "สวัสดีครับคือช่วงนี้ผมนอนไม่ค่อยหลับเวลานอนมันก็จะคิดฟุ้งซ่านตลอดเวลาหรือบางทีก็มีเพลงเล่นในหัวตลอดเวลาเลยทำให้นอนไม่พอ พอจะมีวิธีไหนช่วยจัดการไหมครับ"
# })

# print(my_question_x2)

{'answer': 'The patient is experiencing difficulty sleeping and feelings of restlessness, which may be associated with bipolar disorder or another mood disorder. It is important to seek medical advice for proper diagnosis and treatment.'}
